### Initialization

In [47]:
import numpy as np
import pandas as pd
import sys
import gams.core.numpy
import gamspy as gp

### GAMSPy set-up + introduction

In [ ]:
m=gp.Container()
pd.set_option("mode.copy_on_write", True)


'''
This produces an output-file called dataa_ny.gx for reading into data_from_GR.gms.
This version is heavily handheld for experimentation and debugging
'''

'\nThe data we read in this file is from several different .xlsx sheets. \nThe script runs reasonably efficiently with a few exceptions, it does however rely quite heavily on the pandas-package atm.\nIn terms of robustness, I rely quite heavily on the naming conventions and datastructure in the datasheets.\nThe names are not extracted, but typed which comes with some risk should those conventions change.\nThe gp.Set(), gp.Parameter()-functions that I rely on for datatransfer are also snesitive to changes in the data-structure, that is domain-names must be entered in the "correct" order, especially when relying on the\nvery efficient domain_forwarding for populating sets.\nInitially I define a number of dictionaries. They are mostly self-explanatory, but the ones that are not are adressed immediately below.\nI see no way around having to do this, as the data here is read into sets and parametres that must correpond to sets in the model for it to run.\nFor any other application where dat

### Some dictionaries

In [49]:
'''dictionaries for mapping from raw data onto GR-set members'''
metadata_dicte=pd.read_excel(r'data\metadata.xlsx',sheet_name='energy_products')
dict_e = dict(zip(metadata_dicte['product_greu'], metadata_dicte['product_greu_txt']))

dict_transaction={'transmis_loss':'transmission_losses','cons_inter':'input_in_production','cons_hh':'household_consumption','import':'imports','invent_change':'inventory'}

dict_ebalitems={'ws_marg':'EAV','ret_marg':'DAV','basic':'BASE','co2_xbio':'co2ubio','co2_eq':'co2e','co2_bio':'co2bio'}

dict_a={'tax_products':'TaxSub','tax_vat':'Moms','emp_comp':'SalEmpl','subs_other_production':'OthSubs','tax_other_production':'OthTax','gross_surplus':'OvProd'} #turister

io_inv_dict={'invest_build':'iB','invest_other':'iM','invest_trans':'iT'}

fixed_assets_dict={'N11P':'iM','N1121':'iB','N1122_3':'iB','N1131':'iT','N115':'iM','N117':'iM','N111':'iB'}

dict_io={'38394':'10013','38395':'10012','10010':'10011'}
'''since above is not self-explanatory.
N11P is "ICT-equipment, other machinery and stock and weapons systems"
N1122_3 is "facilities"
N1131 is "means of transport"
N115 is "stock of animals"
N117 is "intellectual rights"
N111 is "housing"
'''
'''List of model years'''
t_list=[i for i in range(1980, 2100)]

### energy_and_emissions.xlsx

### Nonenergyemissions

In [50]:
non_energy_emissions=pd.read_excel(r'data\non_energy_emissions.xlsx',keep_default_na=True)
non_energy_emissions.set_index(['year','bal','flow','indu'],inplace=True)
#stack to obtain a column of emission-types
non_energy_emissions=non_energy_emissions.stack().to_frame(name='level')
non_energy_emissions.dropna(inplace=True)
#impose ebalitems name
non_energy_emissions.index.rename(['year','bal','transaction','d','ebalitems'],inplace=True)
non_energy_emissions.reset_index(inplace=True)


rows_to_replace = non_energy_emissions[non_energy_emissions['d'] == 'cFoodMeat'].copy()
non_energy_emissions= non_energy_emissions[non_energy_emissions['d'] != 'cFoodMeat'] 
# Create new rows for each removed row
new_rows = []
for _, row in rows_to_replace.iterrows():
    for new_d in ['cFoodPig', 'cFoodCow', 'cFoodPoul']:
        new_row = row.copy()
        new_row['d'] = new_d
        new_rows.append(new_row)

# Append the new rows to the DataFrame
non_energy_emissions = pd.concat([non_energy_emissions, pd.DataFrame(new_rows)], ignore_index=True)


#non_energy_emissions.replace({'d':dict_io},inplace=True)
#non_energy_emissions = non_energy_emissions[~non_energy_emissions['d'].isin(['49012', '49022','49025','52000'])]

non_energy_emissions.drop(columns='bal',inplace=True)

non_energy_emissions.replace({'transaction':dict_transaction,'ebalitems':dict_ebalitems},inplace=True)

# set column order
non_energy_emissions = non_energy_emissions[['ebalitems', 'transaction', 'd', 'year', 'level']]

non_energy_emissions19=non_energy_emissions.copy()
non_energy_emissions18=non_energy_emissions.copy()
non_energy_emissions17=non_energy_emissions.copy()
non_energy_emissions16=non_energy_emissions.copy()
non_energy_emissions15=non_energy_emissions.copy()
non_energy_emissions14=non_energy_emissions.copy()
non_energy_emissions13=non_energy_emissions.copy()
non_energy_emissions12=non_energy_emissions.copy()
non_energy_emissions19['year']='2019'
non_energy_emissions18['year']='2018'
non_energy_emissions17['year']='2017'
non_energy_emissions16['year']='2016'
non_energy_emissions15['year']='2015'
non_energy_emissions14['year']='2014'
non_energy_emissions13['year']='2013'
non_energy_emissions12['year']='2012'
non_energy_emissions=pd.concat([non_energy_emissions,non_energy_emissions19,non_energy_emissions18,non_energy_emissions17,non_energy_emissions16,non_energy_emissions15,non_energy_emissions14,non_energy_emissions13,non_energy_emissions12])



### Emissions

In [51]:
energy_and_emissions=pd.read_excel(r'data\energy_and_emissions.xlsx',keep_default_na=True)

'''rename coslumns for compatibility with data loading'''
energy_and_emissions.rename(columns={'indu':'d','product':'e','purp':'es','flow':'transaction'},inplace=True)

#energy_and_emissions.replace({'d':dict_io},inplace=True)
#energy_and_emissions =energy_and_emissions[~energy_and_emissions['d'].isin(['49012', '49022','49025','52000'])]


rows_to_replace = energy_and_emissions[energy_and_emissions['d'] == 'cFoodMeat'].copy()
energy_and_emissions=energy_and_emissions[energy_and_emissions['d'] != 'cFoodMeat'] 
# Create new rows for each removed row
new_rows = []
for _, row in rows_to_replace.iterrows():
    for new_d in ['cFoodPig', 'cFoodCow', 'cFoodPoul']:
        new_row = row.copy()
        new_row['d'] = new_d
        new_rows.append(new_row)

# Append the new rows to the DataFrame
energy_and_emissions = pd.concat([energy_and_emissions, pd.DataFrame(new_rows)], ignore_index=True)



energy_and_emissions.set_index(['year','bal','transaction','d','es','e'],drop=True,inplace=True)

energy_and_emissions=energy_and_emissions.stack().to_frame(name='level')
energy_and_emissions.index.set_names(['year','bal','transaction','d','es','e','ebalitems'],inplace=True)
energy_and_emissions.reset_index(inplace=True)
energy_and_emissions.fillna({'es':'unspecified'},inplace=True)
energy_and_emissions.replace({'transaction':dict_transaction,'ebalitems':dict_ebalitems},inplace=True)


'''replace those pesky nans with a target-specific label'''
energy_and_emissions['d'] = energy_and_emissions.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['transaction'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['transaction'] == 'inventory' 
                                     else ('tl' if pd.isna(row['d']) and row['transaction'] == 'transmission_losses' 
                                           else ('natural_input' if pd.isna(row['d']) and row['transaction'] == 'nat_input'
                                                 else ('residual' if pd.isna(row['d']) and row['transaction'] == 'res_input'
                                                       else ('19000' if pd.isna(row['d']) and row['transaction']=='imports'
                                                            else row['d']))))), axis=1)

energy_and_emissions['e']=energy_and_emissions['e'].replace(dict_e)

# set column order
energy_and_emissions = energy_and_emissions[['ebalitems', 'transaction', 'd', 'es', 'e', 'year', 'level']]
'''retrieve unique values from the e-column to populate e and its superset out.
I add some records manually that are explicitly called in the model, but are not present in data.
'''
e_vals=list(set(energy_and_emissions[['e']].values.flatten()))
out_vals=e_vals.copy()
out_vals.extend(['out_other','WholeAndRetailSaleMarginE','Natural gas (Extraction)'])





energy_and_emissions19=energy_and_emissions.copy()
energy_and_emissions18=energy_and_emissions.copy()
energy_and_emissions17=energy_and_emissions.copy()
energy_and_emissions16=energy_and_emissions.copy()
energy_and_emissions15=energy_and_emissions.copy()
energy_and_emissions14=energy_and_emissions.copy()
energy_and_emissions13=energy_and_emissions.copy()
energy_and_emissions12=energy_and_emissions.copy()
energy_and_emissions19['year']='2019'
energy_and_emissions18['year']='2018'
energy_and_emissions17['year']='2017'
energy_and_emissions16['year']='2016'
energy_and_emissions15['year']='2015'
energy_and_emissions14['year']='2014'
energy_and_emissions13['year']='2013'
energy_and_emissions12['year']='2012'
energy_and_emissions=pd.concat([energy_and_emissions,energy_and_emissions19,energy_and_emissions18,energy_and_emissions17,energy_and_emissions16,energy_and_emissions15,energy_and_emissions14,energy_and_emissions13,energy_and_emissions12])



In [52]:
energy_and_emissions_base=energy_and_emissions.loc[energy_and_emissions['ebalitems']=='BASE']
energy_and_emissions_base_02000=energy_and_emissions_base.loc[energy_and_emissions_base['d']=='01080']
energy_and_emissions_base_02000_noh_transp=energy_and_emissions_base_02000.loc[(energy_and_emissions_base_02000['es']!='heating') &(energy_and_emissions_base_02000['es']!='transport')]
energy_and_emissions_base_02000_noh_transp

,ebalitems,transaction,d,es,e,year,level


In [53]:
energy_and_emissions_base=energy_and_emissions.loc[energy_and_emissions['ebalitems']=='pj']
energy_and_emissions_base_02000=energy_and_emissions_base.loc[energy_and_emissions_base['d']=='01080']
energy_and_emissions_base_02000_noh_transp=energy_and_emissions_base_02000.loc[(energy_and_emissions_base_02000['es']!='heating') &(energy_and_emissions_base_02000['es']!='transport')]
energy_and_emissions_base_02000_noh_transp

,ebalitems,transaction,d,es,e,year,level


### IO-loading

In [54]:
'''gampy does not support domain forwarding from set to set.
To construct a superset of IO-sectors we must initially populate the superset, then provide it as domain for subsets, which we can then populate using the tried and tested domain_forwarding method.
To populate the superset, we must load data from the io-spreadsheets.
On the data:
The Danish IO-tables are received in long-format and matrix-format. I read the long-format for convenience, but the content of the data is better understood when veiwing in matrix-format.
The columns consist of demand-components, which can be either sectors of the economy, export, investment or some final-demand component such as food or housing.
The rows consist of inputs, and can be subdivided into three categories, domestic production, import and primary inputs.
The primary inputs are taxes, subsidies, employee compensation and the likes.
Imported and domestically produced inputs are industry-outputs such that a number in any entry of the IO-matrix can be read as the supply from the row-index to the column-index.
To make this data compatible with the model, we must first inform the model how to interpret the indices. We do this by defining sets.
For instance the set "i" is the set of sectors in the model, we can therefore say that the rows of the import- and domestically produced part of the io-table, consist of members of i. 
The set d is the set of demand-components and corresponds to the columns of the io-tables.
By creating separate variables for imported and domestically produced supply from the io-table, we can define a variable like vIO_y, and define in on [i,d,t], which we can then read as 
domestic sector i's supply to demand component d at year t. Atm, I think the easiest to follow in the extremely likely event that data is not in the exact same format, is the current version
in which I explicitly label columns in the dataframes according to their GR-set-counterparts. 
It is important not to mess with the sets, as they are called explicitly in the model, always by name and on occasion we also refer to specific elements.
The consequence of this is that names have to be inserted here. This will almost certainly need to be edited for other sources of data.
I try to make apparent from the naming throughout where columns and rows are meant to end up in the model which
I suspect will ultimately be more convenient for adapting this code to other datasets than attempting to accomplish the
highest possible degree of automization.
'''
#energy
io_energy=pd.read_excel(r'data\io_energy_long_format.xlsx',keep_default_na=True)
io_energy_forlater=io_energy.copy(deep=True)
io_energy.rename(columns={'row_l2':'i','col_l1':'DELETE','col_l2':'d','value':'level'},inplace=True)
#io_energy.replace({'i':dict_io},inplace=True)
#io_energy.replace({'d':dict_io},inplace=True)
#io_energy = io_energy[~io_energy['i'].isin(['49012', '49022','49025','52000']) & ~io_energy['d'].isin(['49012', '49022','49025','52000'])]


rows_to_replace = io_energy[io_energy['d'] == 'cFoodMeat'].copy()
io_energy=io_energy[io_energy['d'] != 'cFoodMeat'] 
# Create new rows for each removed row
new_rows = []
for _, row in rows_to_replace.iterrows():
    for new_d in ['cFoodPig', 'cFoodCow', 'cFoodPoul']:
        new_row = row.copy()
        new_row['d'] = new_d
        new_rows.append(new_row)

# Append the new rows to the DataFrame
io_energy = pd.concat([io_energy, pd.DataFrame(new_rows)], ignore_index=True)

#regular
io=pd.read_excel(r'data\io_long_format.xlsx')
io_forlater=io.copy(deep=True)
io.rename(columns={'row_l2':'i','col_l1':'DELETE','col_l2':'d','value':'level'},inplace=True)
#io.replace({'i':dict_io},inplace=True)
#io.replace({'d':dict_io},inplace=True)
#io = io[~io['i'].isin(['49012', '49022','49025','52000']) & ~io['d'].isin(['49012', '49022','49025','52000'])]


rows_to_replace = io[io['d'] == 'cFoodMeat'].copy()
io=io[io['d'] != 'cFoodMeat'] 
# Create new rows for each removed row
new_rows = []
for _, row in rows_to_replace.iterrows():
    for new_d in ['cFoodPig', 'cFoodCow', 'cFoodPoul']:
        new_row = row.copy()
        new_row['d'] = new_d
        new_rows.append(new_row)

# Append the new rows to the DataFrame
io = pd.concat([io, pd.DataFrame(new_rows)], ignore_index=True)

'''fill nans based on previous entries. In Danish dataset, col_l1 contains supercategories for col_l2.
Meanwhile, we use col_l2, which contains useful categories such as sectors (as opposed to cons_inter). 
Categories which do not have a subcategory, such as cons_publ (public consumption) have an empty cell at col_l2.
In this program I create the set of demand components d from its members, so in order to represent the demand components that do
not have multiple subcategories, I run the lines below which fills the appropriate GR-demand component based on the supercategory, i.e.
if row10 supercategory is export, replace NaN in the demand component column of row10 with xOth.
'''
io['d'] = io.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['DELETE'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['DELETE'] == 'invent_change' 
                                     else ('g' if pd.isna(row['d']) and row['DELETE'] == 'cons_publ' 
                                           else ('iB' if pd.isna(row['d']) and row['DELETE'] == 'invest_build'
                                                 else ('iT' if pd.isna(row['d']) and row['DELETE'] == 'invest_trans'
                                                       else ('iM' if pd.isna(row['d']) and row['DELETE']=='invest_other'
                                                            else row['d']))))), axis=1)

io_energy['d'] = io_energy.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['DELETE'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['DELETE'] == 'invent_change' 
                                     else ('g' if pd.isna(row['d']) and row['DELETE'] == 'cons_publ' 
                                           else ('iB' if pd.isna(row['d']) and row['DELETE'] == 'invest_build'
                                                 else ('iT' if pd.isna(row['d']) and row['DELETE'] == 'invest_trans'
                                                       else ('iM' if pd.isna(row['d']) and row['DELETE']=='invest_other'
                                                            else row['d']))))), axis=1)

'''Editors note: At the time of writing, the model is not equipped to handle the distinction between con_hh and cons_hh_foreign, therefore
for the time being we simply add them together, I do this by creating a boolean mask to identify rows where the aforementioned supercategory-column 
contains the string cons_hh, which in the GR dataset is both cons_hh and cons_hh_foreign_tou. I then build a dataframe of the entries with only the members of
the original frame(s) that has has cons_hh or cons_hh_foreign_tou as supercategory. I then group entries in this dataset based on entries in the other columns (except of course level) and add them together.
The rows that were selected for this process is then dropped from the original frame(s) and the aggregated rows are added back.
'''
mask=io["DELETE"].str.contains("cons_hh")
mask_ene = io_energy["DELETE"].str.contains("cons_hh")
io_agg = io[mask].groupby(io.columns.difference(["DELETE","level"]).tolist(), as_index=False)["level"].sum()
io_energy_agg= io_energy[mask_ene].groupby(io_energy.columns.difference(["DELETE","level"]).tolist(), as_index=False)["level"].sum()
'''drop rows that had been aggregated'''
io = io[~mask]
io_energy=io_energy[~mask_ene]
'''drop DELETE'''
io.drop(columns=['DELETE'],inplace=True)
io_energy.drop(columns=['DELETE'],inplace=True)
'''reconcat'''
io=pd.concat([io,io_agg])
io_energy=pd.concat([io_energy,io_energy_agg])
'''add t's'''
io=pd.concat([io,pd.DataFrame(columns=['t'],index=io.index,data=2020)],axis=1)
io_energy=pd.concat([io_energy,pd.DataFrame(columns=['t'],index=io_energy.index,data=2020)],axis=1)


'''try yrs'''
io_19=io.copy()
io_18=io.copy()
io_17=io.copy()
io_16=io.copy()
io_15=io.copy()
io_14=io.copy()
io_13=io.copy()
io_12=io.copy()
io_19['t']='2019'
io_18['t']='2018'
io_17['t']='2017'
io_16['t']='2016'
io_15['t']='2015'
io_14['t']='2014'
io_13['t']='2013'
io_12['t']='2012'
io=pd.concat([io,io_19,io_18,io_17,io_16,io_15,io_14,io_13,io_12])

'''try yrs'''
io_energy19=io_energy.copy()
io_energy18=io_energy.copy()
io_energy17=io_energy.copy()
io_energy16=io_energy.copy()
io_energy15=io_energy.copy()
io_energy14=io_energy.copy()
io_energy13=io_energy.copy()
io_energy12=io_energy.copy()
io_energy19['t']='2019'
io_energy18['t']='2018'
io_energy17['t']='2017'
io_energy16['t']='2016'
io_energy15['t']='2015'
io_energy14['t']='2014'
io_energy13['t']='2013'
io_energy12['t']='2012'
io_energy=pd.concat([io_energy,io_energy19,io_energy18,io_energy17,io_energy16,io_energy15,io_energy14,io_energy13,io_energy12])


'''reorder for consistency with GR-variables'''
io=io[['row_l1','i', 'd', 't', 'level']]
io_energy=io_energy[['row_l1','i', 'd', 't', 'level']]
'''disentagle'''
io_y=io[io['row_l1']=='production']
io_m=io[io['row_l1']=='import']
io_a=io[io['row_l1']=='prim_input']
io_ene_y=io_energy[io_energy['row_l1']=='production']
io_ene_m=io_energy[io_energy['row_l1']=='import']
io_ene_a=io_energy[io_energy['row_l1']=='prim_input']
'''drop columns'''
io_y=io_y.drop(columns=['row_l1'])
io_m=io_m.drop(columns=['row_l1'])
io_ene_a=io_ene_a.drop(columns=['row_l1'])
io_ene_y=io_ene_y.drop(columns=['row_l1'])
io_ene_m=io_ene_m.drop(columns=['row_l1'])
io_a=io_a.drop(columns=['row_l1'])
'''apply a_dict'''
io_a.replace({'i':dict_a},inplace=True)
io_ene_a.replace({'i':dict_a},inplace=True)

io_a=io_a.groupby(['d','i','t'],as_index=False).agg({'level':'sum'})
io_combined_a = pd.concat([io_a, io_ene_a]).groupby(io_a.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
'''add energy to non-energy for vIO_{y,m,a}'''
io_combined_y = pd.concat([io_y, io_ene_y]).groupby(io_y.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
io_combined_m=pd.concat([io_m, io_ene_m]).groupby(io_m.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
io_combined_a=io_combined_a.groupby(['d','i','t'],as_index=False).agg({'level':'sum'})
'''adding tot by turning on below. Note, then 'tot' should also be an element of d yabish'''
'''io_y_agg=io_y.groupby('d',as_index=False)['level'].sum()
io_y_agg.insert(0,'i','tot')
io_y_agg.insert(2,'t','2020')
io_y=pd.concat([io_y,io_y_agg])
io_m_agg=io_m.groupby('d',as_index=False)['level'].sum()
io_m_agg.insert(0,'i','tot')
io_m_agg.insert(2,'t','2020')
io_m=pd.concat([io_m,io_m_agg])
io_a_agg=io_a.groupby('d',as_index=False)['level'].sum()
io_a_agg.insert(0,'i','tot')
io_a_agg.insert(2,'t','2020')
io_a=pd.concat([io_a,io_a_agg])
io_combined_y_agg=io_combined_y.groupby('d',as_index=False)['level'].sum()
io_combined_y_agg.insert(0,'i','tot')
io_combined_y_agg.insert(2,'t','2020')
io_combined_y=pd.concat([io_combined_y,io_combined_y_agg])
io_combined_m_agg=io_combined_m.groupby('d',as_index=False)['level'].sum()
io_combined_m_agg.insert(0,'i','tot')
io_combined_m_agg.insert(2,'t','2020')
io_combined_m=pd.concat([io_combined_m,io_combined_m_agg])'''

'''change order for GR-compatibility'''
io_a=io_a[['i','d','t','level']]
io_combined_a=io_combined_a[['i','d','t','level']]

'''A list of elements in i'''
i_elements =list(set(io_y['i']).union(set(io_m['i'])))
i_re_elements=[i+'_re' for i in i_elements]
'''there is another set "rx" whose elements are those of i, then there is a set which serves the purpose of mapping between rx and re.
Below I construct a list of tuples on the form (x,x_re) where x ∈ re to populate this set.
'''
sorted_i_re_elements=sorted(i_re_elements,key=lambda x: i_elements.index(x.split('_')[0]))
rx2re_list=list(zip(i_elements,sorted_i_re_elements))

In [55]:
'''IO investment from here'''

io_inv=pd.read_excel(r'data\io_invest_long_format.xlsx',keep_default_na=True)
io_inv.rename(columns={'col':'i','invest_group':'k','value':'level'},inplace=True)

#io_inv.replace({'i':dict_io},inplace=True)
#io_inv = io_inv[~io_inv['i'].isin(['49012', '49022','49025','52000'])]


'''try yrs'''
io_inv19=io_inv.copy()
io_inv18=io_inv.copy()
io_inv17=io_inv.copy()
io_inv16=io_inv.copy()
io_inv15=io_inv.copy()
io_inv14=io_inv.copy()
io_inv13=io_inv.copy()
io_inv12=io_inv.copy()
io_inv19['t']='2019'
io_inv18['t']='2018'
io_inv17['t']='2017'
io_inv16['t']='2016'
io_inv15['t']='2015'
io_inv14['t']='2014'
io_inv13['t']='2013'
io_inv12['t']='2012'
io_inv=pd.concat([io_inv,io_inv19,io_inv18,io_inv17,io_inv16,io_inv15,io_inv14,io_inv13,io_inv12])


'''apply dict for GR-compatible codes'''
io_inv['k']=io_inv['k'].replace(io_inv_dict)

'''atm we do not care abt. "sender" of capital, just building qI_k_i'''
io_inv_qI_k_i=io_inv.copy(deep=True)

io_inv_qI_k_i.drop(columns=['row_l1','row_l2'],inplace=True)
io_inv_qI_k_i=io_inv_qI_k_i[['k','i','year','level']]
'''aggregate'''
io_inv_qI_k_i_agg=io_inv_qI_k_i.groupby(['k','i','year'],as_index=False).agg({'level':'sum'})


### Currently exporting from herer

### demand-side based io incl. employee compensation

In [56]:

'''build qL'''
io_qRxE=io_forlater[io_forlater['col_l1']=='cons_inter']

io_qRxE=io_qRxE.rename(columns={'col_l2':'i'})

io_qRxE=io_qRxE.replace({'row_l2':dict_a})

io_l=io_qRxE[io_qRxE['row_l2']=='SalEmpl']
io_l_s=io_l.groupby(['i','year'],as_index=False).agg({'value':'sum'})
'''
Above is the total value of labor from employees. 
This must be upscaled by the contribution of independents
Wages of independents are somewhat complicated. We calculate them by:
wages_employed * hours_independents / hours_employed
and add this to the existing wage sum.
Below reads the data required to compute the expression above:
'''

employed_fullset=pd.read_excel(r'data\employed.xlsx',keep_default_na=True)
employed_fullset.rename(columns={'indu':'i'},inplace=True)
employed_employees=employed_fullset[employed_fullset['type']=='employees'][['i', 'hours']]
employed_independent=employed_fullset[employed_fullset['type']=='self-employed'][['i', 'hours']]

wagesum=io_l_s[['i','value']]
# set index
wagesum.set_index('i',inplace=True)
employed_employees.set_index('i',inplace=True)
employed_employees.rename(columns={'hours':'value'},inplace=True)
employed_independent.set_index('i',inplace=True)
employed_independent.rename(columns={'hours':'value'},inplace=True)

#make sure that all indices are valid
wagesum_index=wagesum.index.union(employed_employees.index).union(employed_independent.index)
wagesum =wagesum.reindex(wagesum_index, fill_value=0)
employed_employees=employed_employees.reindex(wagesum_index, fill_value=0)
employed_independent=employed_independent.reindex(wagesum_index, fill_value=0)

#calculate actual wage compensation
wagesum_=wagesum+wagesum*employed_independent/employed_employees
wagesum_.reset_index(inplace=True)
wagesum_.rename(columns={'value':'level'},inplace=True)
#insert t
wagesum_with_t=pd.concat([wagesum_,pd.DataFrame(index=wagesum_.index,columns=['t'],data=2020)],axis=1)
#reorder columns
wagesum_with_t=wagesum_with_t[['i','t','level']]


wagesum_with_t19=wagesum_with_t.copy()
wagesum_with_t18=wagesum_with_t.copy()
wagesum_with_t17=wagesum_with_t.copy()
wagesum_with_t16=wagesum_with_t.copy()
wagesum_with_t15=wagesum_with_t.copy()
wagesum_with_t14=wagesum_with_t.copy()
wagesum_with_t13=wagesum_with_t.copy()
wagesum_with_t12=wagesum_with_t.copy()
wagesum_with_t19['t']='2019'
wagesum_with_t18['t']='2018'
wagesum_with_t17['t']='2017'
wagesum_with_t16['t']='2016'
wagesum_with_t15['t']='2015'
wagesum_with_t14['t']='2014'
wagesum_with_t13['t']='2013'
wagesum_with_t12['t']='2012'
wagesum_with_t=pd.concat([wagesum_with_t,wagesum_with_t19,wagesum_with_t18,wagesum_with_t17,wagesum_with_t16,wagesum_with_t15,wagesum_with_t14,wagesum_with_t13,wagesum_with_t12])
#total number of employed including independents
nemployed_frame = pd.DataFrame(columns=['t', 'level'], data=[[2020, employed_fullset['employed'].sum()]])

nemployed_frame19=nemployed_frame.copy()
nemployed_frame18=nemployed_frame.copy()
nemployed_frame17=nemployed_frame.copy()
nemployed_frame16=nemployed_frame.copy()
nemployed_frame15=nemployed_frame.copy()
nemployed_frame14=nemployed_frame.copy()
nemployed_frame13=nemployed_frame.copy()
nemployed_frame12=nemployed_frame.copy()
nemployed_frame19['t']='2019'
nemployed_frame18['t']='2018'
nemployed_frame17['t']='2017'
nemployed_frame16['t']='2016'
nemployed_frame15['t']='2015'
nemployed_frame14['t']='2014'
nemployed_frame13['t']='2013'
nemployed_frame12['t']='2012'
nemployed_frame=pd.concat([nemployed_frame12,nemployed_frame13,nemployed_frame14,nemployed_frame15,nemployed_frame16,nemployed_frame17,nemployed_frame18,nemployed_frame19,nemployed_frame])

### Capital, fixed assets

In [57]:
fixed_assets=pd.read_excel(r'data\fixed_assets.xlsx',keep_default_na=True)
'''map onto gr-codes'''
fixed_assets.replace({'asset':fixed_assets_dict},inplace=True)

fixed_assets.rename(columns={'asset':'k','indu':'i','value':'level'},inplace=True)
'''reorder columns'''
fixed_assets=fixed_assets[['k','i','year','level']]
'''GR-split of capital is coarser than data, so we must aggregate'''
fixed_assets=fixed_assets.groupby(['k','i','year'],as_index=False).agg({'level':'sum'})


fixed_assets19=fixed_assets.copy()
fixed_assets18=fixed_assets.copy()
fixed_assets17=fixed_assets.copy()
fixed_assets16=fixed_assets.copy()
fixed_assets15=fixed_assets.copy()
fixed_assets14=fixed_assets.copy()
fixed_assets13=fixed_assets.copy()
fixed_assets12=fixed_assets.copy()
fixed_assets19['year']='2019'
fixed_assets18['year']='2018'
fixed_assets17['year']='2017'
fixed_assets16['year']='2016'
fixed_assets15['year']='2015'
fixed_assets14['year']='2014'
fixed_assets13['year']='2013'
fixed_assets12['year']='2012'
fixed_assets=pd.concat([fixed_assets12,fixed_assets13,fixed_assets14,fixed_assets15,fixed_assets16,fixed_assets17,fixed_assets18,fixed_assets19,fixed_assets])



### ets.xlsx

In [58]:
ets=pd.read_excel(r'data\ets.xlsx',keep_default_na=True)
#reorder columns for free allowances and drop redundants
qCO2_ETS_freeallowances=ets[['indu','year', 'free_allowances']]
#level
qCO2_ETS_freeallowances.rename(columns={'free_allowances':'level','indu':'i'},inplace=True)



qCO2_ETS_freeallowances19=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances18=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances17=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances16=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances15=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances14=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances13=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances12=qCO2_ETS_freeallowances.copy()
qCO2_ETS_freeallowances19['year']='2019'
qCO2_ETS_freeallowances18['year']='2018'
qCO2_ETS_freeallowances17['year']='2017'
qCO2_ETS_freeallowances16['year']='2016'
qCO2_ETS_freeallowances15['year']='2015'
qCO2_ETS_freeallowances14['year']='2014'
qCO2_ETS_freeallowances13['year']='2013'
qCO2_ETS_freeallowances12['year']='2012'
qCO2_ETS_freeallowances=pd.concat([qCO2_ETS_freeallowances12,qCO2_ETS_freeallowances13,qCO2_ETS_freeallowances14,qCO2_ETS_freeallowances15,qCO2_ETS_freeallowances16,qCO2_ETS_freeallowances17,qCO2_ETS_freeallowances18,qCO2_ETS_freeallowances19,qCO2_ETS_freeallowances])

qCO2_ETS_freeallowances


,i,year,level
0,01020,2012,8.790
1,0600a,2012,1114.711
2,10010,2012,21.019
3,10020,2012,82.599
4,10040,2012,45.242
...,...,...,...
14,35002,2020,3.947
15,35011,2020,1486.985
16,38393,2020,350.864
17,51001,2020,333.866


### emissions_brigde_items.xlsx

In [59]:
emissions_bridge_items=pd.read_excel(r'data\emissions_brigde_items.xlsx',keep_default_na=True)

qEmmLULUCF = emissions_bridge_items.loc[emissions_bridge_items['item'] == 'lulucf', ['year','co2_eq']]

'''this is pretty fun.
Year is stored as a floating point number 2020.0 - which is not the same as the string 2020.
When exporting without converting to string, this then causes gamspy to look for an element corresponding to the floating point number 2020.0 in t, which it will not find.
One can then ask: Why did this not happen when we loaded nEmployed?
Because when we loaded nEmployed, we actually found it more convenient to construct a dataframe from scratch and populate with the members 2020 and some column sum from the data.
'''
qEmmLULUCF['year'] = qEmmLULUCF['year'].astype('string')

qEmmLULUCF.rename(columns={'co2_eq':'level'},inplace=True)

qEmmLULUCF19=qEmmLULUCF.copy()
qEmmLULUCF18=qEmmLULUCF.copy()
qEmmLULUCF17=qEmmLULUCF.copy()
qEmmLULUCF16=qEmmLULUCF.copy()
qEmmLULUCF15=qEmmLULUCF.copy()
qEmmLULUCF14=qEmmLULUCF.copy()
qEmmLULUCF13=qEmmLULUCF.copy()
qEmmLULUCF12=qEmmLULUCF.copy()
qEmmLULUCF19['year']='2019'
qEmmLULUCF18['year']='2018'
qEmmLULUCF17['year']='2017'
qEmmLULUCF16['year']='2016'
qEmmLULUCF15['year']='2015'
qEmmLULUCF14['year']='2014'
qEmmLULUCF13['year']='2013'
qEmmLULUCF12['year']='2012'
qEmmLULUCF=pd.concat([qEmmLULUCF12,qEmmLULUCF13,qEmmLULUCF14,qEmmLULUCF15,qEmmLULUCF16,qEmmLULUCF17,qEmmLULUCF18,qEmmLULUCF19,qEmmLULUCF])



emissions_bridge_items_bordertrade=emissions_bridge_items.loc[emissions_bridge_items['item']=='bord_trade']
emissions_bridge_items_bordertrade.rename(columns=dict_ebalitems,inplace=True)
emissions_bridge_items_bordertrade=emissions_bridge_items_bordertrade.dropna(axis=1)
#drop item
emissions_bridge_items_bordertrade.drop(columns=['item'],inplace=True)
#stack
emissions_bridge_items_bordertrade.set_index('year',inplace=True)
emissions_bridge_items_bordertrade=emissions_bridge_items_bordertrade.stack().to_frame(name='level').reset_index()
#reorder columns
emissions_bridge_items_bordertrade=emissions_bridge_items_bordertrade[['level_1','year','level']]

emissions_bridge_items_bordertrade17=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade19=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade18=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade16=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade15=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade14=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade13=emissions_bridge_items_bordertrade.copy()
emissions_bridge_items_bordertrade12=emissions_bridge_items_bordertrade.copy()

emissions_bridge_items_bordertrade19['year']='2019'
emissions_bridge_items_bordertrade18['year']='2018'
emissions_bridge_items_bordertrade17['year']='2017'
emissions_bridge_items_bordertrade16['year']='2016'
emissions_bridge_items_bordertrade15['year']='2015'
emissions_bridge_items_bordertrade14['year']='2014'
emissions_bridge_items_bordertrade13['year']='2013'
emissions_bridge_items_bordertrade12['year']='2012'
emissions_bridge_items_bordertrade=pd.concat([emissions_bridge_items_bordertrade12,emissions_bridge_items_bordertrade13,emissions_bridge_items_bordertrade14,emissions_bridge_items_bordertrade15,emissions_bridge_items_bordertrade16,emissions_bridge_items_bordertrade17,emissions_bridge_items_bordertrade19,emissions_bridge_items_bordertrade18,emissions_bridge_items_bordertrade])
emissions_bridge_items_bordertrade

,level_1,year,level
0,ch4,2012,0.025
1,co2ubio,2012,-257.000
2,n2o,2012,-0.009
3,co2e,2012,-258.685
0,ch4,2013,0.025
1,co2ubio,2013,-257.000
2,n2o,2013,-0.009
3,co2e,2013,-258.685
0,ch4,2014,0.025
1,co2ubio,2014,-257.000


### government_finances.xlsx

In [60]:
#government_finances=pd.read_excel(r'data\government_finances.xlsx',keep_default_na=True)

'''Note:
In GR, these values come from MAKRO.
This makes it somewhat difficult to make sense of any deviations and/or constructed variables.
Ask, if you find some bigguns
'''
'''
#convert to string
government_finances['year']=government_finances['year'].astype('string')
#value2level
government_finances.rename(columns={'value':'level'},inplace=True)
#transfers to abroad
government_finances_transfertorow=government_finances.loc[government_finances['trans']=='transfer_to_row']
vGov2Foreign=government_finances_transfertorow[['year','level']]
#transfers from abroaD
government_finances_transferfromrow=government_finances.loc[government_finances['trans']=='transfers_from_row']
vGovReceiveF=government_finances_transferfromrow[['year','level']]
#Land rent
government_finances_rent=government_finances.loc[government_finances['trans']=='rent']
vGovRent=government_finances_rent[['year','level']]
#government investments
government_finances_invest=government_finances.loc[government_finances['trans']=='invest']
vGovInv=government_finances_invest[['year','level']]
#government subsidies
government_finances_subsidies=government_finances.loc[government_finances['trans']=='subs']
vGovSub=government_finances_subsidies[['year','level']]
#capital transfers to domestic sectors
government_finances_transferstofirms=government_finances.loc[government_finances['trans']=='cap_transfer_to_dom']
vGov2Firms=government_finances_transferstofirms[['year','level']]
#capital transfers from domestic firms
government_finances_transfersfromfirms=government_finances.loc[government_finances['trans']=='cap_transfers_from_dom']
vGovReceiveFirms=government_finances_transfersfromfirms[['year','level']]
#Public expenditures, not including those paid by EU
government_finances_exp = government_finances.loc[(government_finances['balance']=='exp') & (government_finances['trans']!='interest')]
vGovExp=government_finances_exp[['year','level']]
vGovExp=vGovExp.groupby(['year'],as_index=False).agg({'level':'sum'})
#Public revenues (not including interests)
government_finances_rev=government_finances.loc[(government_finances['balance']=='rev') &(government_finances['trans']!='interest')]
vGovRev=government_finances_rev[['year','level']]
vGovRev=vGovRev.groupby(['year'],as_index=False).agg({'level':'sum'})
#Revenue from income taxation (kildeskatter)
government_finances_source=government_finances.loc[government_finances['trans']=='tax_direct_source']
vtSource=government_finances_source[['year','level']]
#VAT
government_finances_vat=government_finances.loc[government_finances['trans']=='tax_indirect_vat']
vtVAT=government_finances_vat[['year','level']]
#media tax¨
government_finances_media=government_finances.loc[government_finances['trans']=='tax_direct_media']
vtMedia=government_finances_media[['year','level']]
#vehicles
government_finances_vehicles=government_finances.loc[government_finances['trans']=='tax_direct_vehicles']
vtCarWeight=government_finances_vehicles[['year','level']]
#Revenue from indirect taxes (sum)
government_finances_indirect=government_finances.loc[(government_finances['balance']=='rev') & (government_finances['trans'].str.contains('tax_indirect'))]
vtIndirect=government_finances_indirect[['year','level']]
vtIndirect=vtIndirect.groupby(['year'],as_index=False).agg({'level':'sum'})
#Revenue from direct taxes (sum)
government_finances_direct=government_finances.loc[(government_finances['trans'].str.contains('tax_direct'))&(government_finances['balance']=='rev')]
vtDirect=government_finances_direct[['year','level']]
vtDirect=vtDirect.groupby(['year'],as_index=False).agg({'level':'sum'})
#Rest
government_finances_other=government_finances.loc[(government_finances['balance']=='rev')&(~government_finances['trans'].str.contains('tax_direct'))&(~government_finances['trans'].str.contains('tax_indirect'))]
vGovRevRest=government_finances_other[['year','level']]
vGovRevRest=vGovRevRest.groupby(['year'],as_index=False).agg({'level':'sum'})
#final public consumption
government_finances_final=government_finances.loc[government_finances['trans']=='cons_publ']
vG=government_finances_final[['year','level']]
#gov transfers
government_finances_transfer=government_finances.loc[government_finances['trans']=='transfer_to_hh']
vTrans=government_finances_transfer[['year','level']]
# #social contributions
government_finances_social=government_finances.loc[government_finances['trans']=='soc_cont']
vCont=government_finances_social[['year','level']]
#revenue from corporate taxation
government_finances_corp=government_finances.loc[government_finances['trans']=='tax_direct_corp']
vtCorp=government_finances_corp[['year','level']]
#tax on pension
government_finances_pension=government_finances.loc[government_finances['trans']=='tax_direct_pension']
vtPAL=government_finances_pension[['year','level']]
'''
'''To split taxes on personal income, we use the disaggregated sheet from the same xlsx-file.
Obviously one can just look at it and recognize the numbers in the aggregated sheet and load directly therefrom, but since
they are indistinguishable - in all other ways than the value in the aggregated form, I read from the diaggregated sheet for inspectability.
Here I am forced to use the trans_txt-column to distinguish between the personal income taxes since they are otherwise identically labelled
'''
'''
#tax on labour
government_finances_disagg=pd.read_excel(r'data\government_finances.xlsx',sheet_name='gov_fin_disagg',keep_default_na=True)
government_finances_disagg.rename(columns={'value':'level'},inplace=True)
government_finances_disagg['year']=government_finances_disagg['year'].astype('string')
#revenue from contribution to labour market fund
government_finances_taxlaborAM=government_finances_disagg.loc[government_finances_disagg['trans_txt'].str.contains('labour market fund')&(government_finances_disagg['trans']=='tax_direct_other_labor')]
vtAM=government_finances_taxlaborAM[['year','level']]
#other personal income taxes
government_finances_taxlaboroth=government_finances_disagg.loc[(government_finances_disagg['trans']=='tax_direct_other_labor')&(government_finances_disagg['trans_txt'].str.contains('other'))]
vtPersIncRest=government_finances_taxlaboroth[['year','level']]'''

<>:87: SyntaxWarning: invalid escape sequence '\g'
<>:87: SyntaxWarning: invalid escape sequence '\g'
C:\Users\B247549\AppData\Local\Temp\ipykernel_9956\1467318578.py:87: SyntaxWarning: invalid escape sequence '\g'
  '''


"\n#tax on labour\ngovernment_finances_disagg=pd.read_excel(r'data\\government_finances.xlsx',sheet_name='gov_fin_disagg',keep_default_na=True)\ngovernment_finances_disagg.rename(columns={'value':'level'},inplace=True)\ngovernment_finances_disagg['year']=government_finances_disagg['year'].astype('string')\n#revenue from contribution to labour market fund\ngovernment_finances_taxlaborAM=government_finances_disagg.loc[government_finances_disagg['trans_txt'].str.contains('labour market fund')&(government_finances_disagg['trans']=='tax_direct_other_labor')]\nvtAM=government_finances_taxlaborAM[['year','level']]\n#other personal income taxes\ngovernment_finances_taxlaboroth=government_finances_disagg.loc[(government_finances_disagg['trans']=='tax_direct_other_labor')&(government_finances_disagg['trans_txt'].str.contains('other'))]\nvtPersIncRest=government_finances_taxlaboroth[['year','level']]"

### institutional_financial_accounts

In [61]:
'''
institutional_financial_accounts=pd.read_excel(r'data\institutional_financial_accounts.xlsx',keep_default_na=True)
#make sure year is string
institutional_financial_accounts['year']=institutional_financial_accounts['year'].astype('string')

#gov interests
vGovInterest=institutional_financial_accounts.loc[(institutional_financial_accounts['var']=='vNetInterests')&(institutional_financial_accounts['sector']=='gov')]
#net
vGovNetInterest=vGovInterest[['year','net']]
vGovNetInterest.rename(columns={'net':'level'})
#assets
vInterestGovAssets=vGovInterest[['year','as']]
vInterestGovAssets.rename(columns={'as':'level'})
#debt
vInterestGovDebt=vGovInterest[['year','li']]
vInterestGovDebt.rename(columns={'as':'level'})'
'''

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\B247549\AppData\Local\Temp\ipykernel_9956\3066609460.py:1: SyntaxWarning: invalid escape sequence '\i'
  '''


"\ninstitutional_financial_accounts=pd.read_excel(r'data\\institutional_financial_accounts.xlsx',keep_default_na=True)\n#make sure year is string\ninstitutional_financial_accounts['year']=institutional_financial_accounts['year'].astype('string')\n\n#gov interests\nvGovInterest=institutional_financial_accounts.loc[(institutional_financial_accounts['var']=='vNetInterests')&(institutional_financial_accounts['sector']=='gov')]\n#net\nvGovNetInterest=vGovInterest[['year','net']]\nvGovNetInterest.rename(columns={'net':'level'})\n#assets\nvInterestGovAssets=vGovInterest[['year','as']]\nvInterestGovAssets.rename(columns={'as':'level'})\n#debt\nvInterestGovDebt=vGovInterest[['year','li']]\nvInterestGovDebt.rename(columns={'as':'level'})'\n"

In [62]:
non_energy_emissions

,ebalitems,transaction,d,year,level
0,ch4,input_in_production,01011,2020,0.089383
1,co2ubio,input_in_production,01011,2020,239.902860
2,n2o,input_in_production,01011,2020,13.902021
3,co2e,input_in_production,01011,2020,3926.441155
4,ch4,input_in_production,01012,2020,0.005762
...,...,...,...,...,...
166,co2e,input_in_production,off,2012,22.125902
167,ch4,household_consumption,cHou,2012,0.345261
168,co2ubio,household_consumption,cHou,2012,65.464583
169,n2o,household_consumption,cHou,2012,0.010633


### Reading metadata for sets + set_txt

In [63]:
#populate c and add text
metadata_cons_hh=pd.read_excel(r'data\metadata.xlsx',sheet_name='cons_hh',keep_default_na=True)
c_records = list(metadata_cons_hh.itertuples(index=False, name=None))
#populate i and re (with text)
metadata_industries=pd.read_excel(r'data\metadata.xlsx',sheet_name='industries',keep_default_na=True)
i_records = list(metadata_industries.itertuples(index=False, name=None))
i_records_fortot=i_records.copy()
re_records = [(str(x) + '_re', y) for x, y in i_records]
#populate es (w. text)
metadata_energy_purposes=pd.read_excel(r'data\metadata.xlsx',sheet_name='energy_purposes',keep_default_na=True)
es_records = list(metadata_energy_purposes.itertuples(index=False, name=None))
#ppulate a_rows_ (w. text)
metadata_flows=pd.read_excel(r'data\metadata.xlsx',sheet_name='flows',keep_default_na=True)
metadata_flows_a=metadata_flows[metadata_flows['flow_type']=='prim_input']
metadata_flows_a['flow']=metadata_flows_a['flow'].replace(dict_a)
a_records=list(metadata_flows_a[['flow','flow_txt']].itertuples(index=False, name=None))
#populate k (w. text)
metadata_flows_k=metadata_flows[metadata_flows['flow'].str.contains('invest')]
metadata_flows_k['flow']=metadata_flows_k['flow'].replace(io_inv_dict)
k_records=list(metadata_flows_k[['flow','flow_txt']].itertuples(index=False, name=None))
k_records_fortot=k_records.copy()
'''
populate ebalitems, currently 'EAFG_tax is called explicitly in the model and is not present in data, so I add it manually to the set ebalitems and the subset etaxes.
'''
ebalitems_records=list(set(non_energy_emissions['ebalitems']).union(set(energy_and_emissions['ebalitems'])))
ebalitems_records.append('EAFG_tax')
#etaxes records
etaxes_records=[s for s in ebalitems_records if '_tax' in s]

d_records=list(set(io_combined_y['d']).union(set(io_combined_m['d']),set(io_y['d']),set(io_m['d']),set(io_combined_a['d']),set(io_a['d']),set(non_energy_emissions['d']),set(energy_and_emissions['d']),set(i_re_elements)))

In [64]:
c_records=[x for x in d_records if 'c' in x]
c_records

['cFoodVeg',
 'cHou',
 'cCarEne',
 'cHouEne',
 'cFoodFish',
 'cSer',
 'cFoodCow',
 'cCar',
 'cNonFood',
 'cFoodPoul',
 'cFoodBev',
 'cFoodPig',
 'cTou',
 'cFoodDairy']

### GAMS export

In [65]:
'''transaction=gp.Set(m,name='transaction',description='set of transaction types')
t=gp.Set(m,'t',description='year',records=t_list)
t1=gp.Set(m,'t1',domain=[t],description='t1',is_singleton=True,records=['2020'])
es=gp.Set(m,'es',description='energy service')
out=gp.Set(m,'out',description='output types')
a_rows_=gp.Set(m,'a_rows_',description='other rows in the input-output table')
ebalitems=gp.Set(m,'ebalitems',description='identifiers tax joules prices etc for energy components by demand components')
d=gp.Set(m,'d',description='demand components',records=list(set(io_combined_y['d']).union(set(io_combined_m['d']),set(io_y['d']),set(io_m['d']),set(io_combined_a['d']),set(io_a['d']),set(non_energy_emissions['d']),set(energy_and_emissions['d']),set(i_re_elements))))
i=gp.Set(m,'i',description='sectors',records=i_elements)

''''''non-energy emissions''''''
non_energy_emissions=gp.Parameter(m,name='NonEnergyEmissions',domain=[ebalitems,transaction,d,t],description='emission from consumption of non-energy',records=non_energy_emissions.values.tolist(),domain_forwarding=True)

''''''energy emissions''''''
EnergyBalance=gp.Parameter(m,'EnergyBalance',domain=[ebalitems,transaction,d,es,out,t],description='Main data input with regards to energy and energy-related emissions',records=energy_and_emissions[['ebalitems','transaction','d','es','e','year','level']].values.tolist(),domain_forwarding=True)
''''''demand_transaction ⊂ transaction, transaction is currently populated using domain_forwarding, meaning it is not populated before EnergyBalance and NonEnergyEmissions are defined''''''
demand_transaction=gp.Set(m,name='demand_transaction',domain=[transaction],description='Demand components',records=['production','input_in_production','export','inventory','transmission_losses'])
vIO_y=gp.Parameter(m,name='vIO_y',domain=[d,d,t],description='Production IO',records=io_combined_y[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIO_m=gp.Parameter(m,name='vIO_m',domain=[d,d,t],description='Production IO',records=io_combined_m[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)


vIOxE_y=gp.Parameter(m,name='vIOxE_y',domain=[d,d,t],description='non-energy IO of domestic production',records=io_y[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIOxE_m=gp.Parameter(m,name='vIOxE_m',domain=[d,d,t],description='non-energy IO of imports',records=io_m[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)

vIO_a=gp.Parameter(m,name='vIO_a',domain=[a_rows_,d,t],description='other IO',records=io_combined_a[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIOxE_a=gp.Parameter(m,name='vIOxE_a',domain=[a_rows_,d,t],description='non energy other IO',records=io_a[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)

m.write('dataa_ny.gdx')'''

"transaction=gp.Set(m,name='transaction',description='set of transaction types')\nt=gp.Set(m,'t',description='year',records=t_list)\nt1=gp.Set(m,'t1',domain=[t],description='t1',is_singleton=True,records=['2020'])\nes=gp.Set(m,'es',description='energy service')\nout=gp.Set(m,'out',description='output types')\na_rows_=gp.Set(m,'a_rows_',description='other rows in the input-output table')\nebalitems=gp.Set(m,'ebalitems',description='identifiers tax joules prices etc for energy components by demand components')\nd=gp.Set(m,'d',description='demand components',records=list(set(io_combined_y['d']).union(set(io_combined_m['d']),set(io_y['d']),set(io_m['d']),set(io_combined_a['d']),set(io_a['d']),set(non_energy_emissions['d']),set(energy_and_emissions['d']),set(i_re_elements))))\ni=gp.Set(m,'i',description='sectors',records=i_elements)\n\nnon-energy emissions\nnon_energy_emissions=gp.Parameter(m,name='NonEnergyEmissions',domain=[ebalitems,transaction,d,t],description='emission from consumption

In [66]:
'''sets'''
transaction=gp.Set(m,name='transaction',description='set of transaction types')
es=gp.Set(m,'es',description='energy service',records=es_records)
out=gp.Set(m,'out',description='output types',records=out_vals)
e=gp.Set(m,'e',domain=[out],description='energy products by industry',records=e_vals)
t=gp.Set(m,'t',description='year',records=t_list)
t1=gp.Set(m,'t1',domain=[t],description='t1',is_singleton=True,records=['2019'])
a_rows_=gp.Set(m,'a_rows_',description='other rows in the input-output table',records=a_records)
k=gp.Set(m,name="k",description='capital types',records=k_records)
'''ebalitems + subsets, some are manually populated since they lack a sufficiently universal identifier in the data'''
ebalitems=gp.Set(m,'ebalitems',description='identifiers tax joules prices etc for energy components by demand components',records=ebalitems_records)
em=gp.Set(m,name='em',domain=[ebalitems],description='emission types',records=['ch4','co2ubio','n2o','co2e','co2bio'])
etaxes=gp.Set(m,name='etaxes',domain=[ebalitems],description='taxes from ebalitems',records=etaxes_records)
'''d + subsets of d'''
d=gp.Set(m,'d',description='demand components',records=d_records)
invt=gp.Set(m,name='invt',domain=[d],description='Inventories',is_singleton=True,records=['invt'])
i=gp.Set(m,name='i',domain=[d],description='sectors',records=i_records)
tl=gp.Set(m,name='tl',domain=[d],description='Transmission losses',is_singleton=True ,records=['tl'])
x=gp.Set(m,name='x',domain=[d],description='export types',records=['xOth'])
g=gp.Set(m,name='g',domain=[d],description='public consumption',records=['g'])
c=gp.Set(m,name='c',domain=[d],description='private consumption groups',records=c_records)
rx=gp.Set(m,name='rx',domain=[d],description='Non-energy intermediate input types, this is just equal to i ATM.',records=i_records)
re=gp.Set(m,name='re',domain=[d],description='Energy intermediate input types',records=re_records)
rx2re=gp.Set(m,name='rx2re',domain=[rx,re],description='map from rx to re',records=rx2re_list)
'''non-energy emissions'''
non_energy_emissions=gp.Parameter(m,name='NonEnergyEmissions',domain=[ebalitems,transaction,d,t],description='emission from consumption of non-energy',records=non_energy_emissions.values.tolist(),domain_forwarding=True)

'''energy emissions'''
EnergyBalance=gp.Parameter(m,'EnergyBalance',domain=[ebalitems,transaction,d,es,out,t],description='Main data input with regards to energy and energy-related emissions',records=energy_and_emissions[['ebalitems','transaction','d','es','e','year','level']].values.tolist(),domain_forwarding=True)
'''demand_transaction ⊂ transaction, transaction is currently populated using domain_forwarding, meaning it is not populated before EnergyBalance and NonEnergyEmissions are defined'''
demand_transaction=gp.Set(m,name='demand_transaction',domain=[transaction],description='Demand components',records=['production','input_in_production','export','inventory','transmission_losses'])
'''IO'''
vIO_y=gp.Parameter(m,name='vIO_y',domain=[d,d,t],description='Production IO',records=io_combined_y[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIO_m=gp.Parameter(m,name='vIO_m',domain=[d,d,t],description='Production IO',records=io_combined_m[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)


vIOxE_y=gp.Parameter(m,name='vIOxE_y',domain=[d,d,t],description='non-energy IO of domestic production',records=io_y[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIOxE_m=gp.Parameter(m,name='vIOxE_m',domain=[d,d,t],description='non-energy IO of imports',records=io_m[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)

vIO_a=gp.Parameter(m,name='vIO_a',domain=[a_rows_,d,t],description='other IO',records=io_combined_a[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)
vIOxE_a=gp.Parameter(m,name='vIOxE_a',domain=[a_rows_,d,t],description='non energy other IO',records=io_a[['i', 'd', 't', 'level']].values.tolist(),domain_forwarding=True)

'''demand side IO'''
qI_k_i=gp.Parameter(m,'qI_k_i',domain=[k,d,t],description='Real capital stock by capital type and industry',records=io_inv_qI_k_i_agg[['k','i','year','level']].values.tolist())
wagesum_with_t=gp.Parameter(m,name='qL',domain=[d,t],description='Wage expenses',records=wagesum_with_t[['i','t','level']].values.tolist())
nemployed=gp.Parameter(m,name='nEmployed',domain=[t],description='Total number of employees including independents',records=nemployed_frame.values.tolist())

'''capital fixed assets'''
fixed_assets=gp.Parameter(m,name='qK',domain=[k,d,t],description='Capital split on types and sectors',records=fixed_assets[['k','i','year','level']].values.tolist())

'''ets'''
qCO2_ETS_freeallowances=gp.Parameter(m,name='qCO2_ETS_freeallowances',domain=[d,t],description='CO2-ETS free allowances',records=qCO2_ETS_freeallowances[['i','year','level']].values.tolist())

'''emissions bridge items'''
qEmmLULUCF=gp.Parameter(m,name='qEmmLULUCF',domain=[t],description='Total LULUCF-emissions',records=qEmmLULUCF.values.tolist())
qEmmBorderTrade=gp.Parameter(m,name='qEmmBorderTrade',domain=[em,t],description='emissions from border trade',records=emissions_bridge_items_bordertrade.values.tolist())

### Hardcoded + export gdx

In [67]:
'''HARD CODED SETS'''
factors_of_production=gp.Set(m,name='factors_of_production',description='factors of production, hardcoded',records=['iM','iB','iT','labor','RxE','machine_energy','transport_energy','heating_energy','refinery_crudeoil','naturalgas_for_distribution','biogas_for_processing'])
em_accounts=gp.Set(m,name='em_accounts',description='Different accounting levels of emissions inventories',records=['GNA','UNFCCC','GNA_lulucf','UNFCCC_lulucf'])
land5=gp.Set(m,name='land5',records=['forest','wetland','grassland','crop','settlement'])
'''sets already made that need tots'''
i_records_fortot.append(('tot','total'))
k_records_fortot.append(('iTot','total'))
i_=gp.Set(m,name='i_',description='sectors, including total',records=i_records_fortot)
k_=gp.Set(m,name='k_',description='capital types including total, excluding inventories',records=k_records_fortot)


### Additions required for base_model

In [68]:
'''13.3.25:
In order for the base-model to run using output from the data processing script, some extra objects whose origin is not explicitly clear in the current data package.
It is also not certain that these objects are part of the long-term plan.
In order to make the model run, I will create these objects here.
'''

'''Object 1 is the set m, which is a subset of i containing "industries with imports". 
This can be interpreted as either industries abroad that produce stuff that we import (corresponding to the rows in the import-section of the IO-table),
or industries that import something from abroad corresponding to the columns.
In our case this does not really matter if we consider energy-products as well as ordinary sector-specific outputs.
If energy is not counted, the former of the two interpretations does indeed give rise to a proper subset. I, however will for the time being consider energy as well
'''

m_=gp.Set(m,name='m',domain=[i],description='industries with imports',records=io_combined_m['i'].unique().tolist())

m.write('dataa_ny.gdx')
